# COMP 551 — Assignment 2

Authors:
 - Bernier, Audréanne
 - Coull-Neveu, Ben
 - Trachsel-Bourbeau, Anjara



Imports

In [3]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
import seaborn as sns
from math import factorial
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Functions & Models

### Useful Functions

### Model Implementations

## Task 1: Linear Regression with Non-Linear Basis Functions

#### Data Generation & Non-Linear Basis Functions

In [5]:
""" 
1. Data Generation: 
    Generate 100 data points sampled from the following non-linear function:
            y(x) = (log x + 1) cos(x) + sin(2x) + ε
    where x is sampled uniformly in the range [0, 10]. And ε is sampled randomly and independently for each new data point from a Gaussian with mean 0 and variance 1 to simulate real-world noisy data. Plot both the clean data and the noisy data.

2. Non-Linear Basis Functions: Fit a linear regression model to the generated data, but instead of using the original features, transform them using Gaussian basis functions. Specifically, use Gaussian basis functions of the form:
             (x−μ)2 φ(x,μ,σ)=exp − σ2
    where μ is the mean of the Gaussian and σ = 1 is the standard deviation. You will use basis functions with
    increasing complexity, ranging from 0 to 45 Gaussians, by varying the number of basis functions D. Please plot
    the Gaussian basis for range of generated data. Note: For μ, use: μi = xmin + xmax −xmin i, i ∈ [0, D − 1]. D−1
    This is the mean for the ith Gaussian.
"""

' \n1. Data Generation: \n    Generate 100 data points sampled from the following non-linear function:\n            y(x) = (log x + 1) cos(x) + sin(2x) + ε\n    where x is sampled uniformly in the range [0, 10]. And ε is sampled randomly and independently for each new data point from a Gaussian with mean 0 and variance 1 to simulate real-world noisy data. Plot both the clean data and the noisy data.\n\n2. Non-Linear Basis Functions: Fit a linear regression model to the generated data, but instead of using the original features, transform them using Gaussian basis functions. Specifically, use Gaussian basis functions of the form:\n            \x12 (x−μ)2\x13 φ(x,μ,σ)=exp − σ2\n    where μ is the mean of the Gaussian and σ = 1 is the standard deviation. You will use basis functions with\n    increasing complexity, ranging from 0 to 45 Gaussians, by varying the number of basis functions D. Please plot\n    the Gaussian basis for range of generated data. Note: For μ, use: μi = xmin + xmax 

## Task 2: Bias-Variance Tradeoff with Multiple Fits

## Task 3: Regularization with Cross-Validation

## Task 4: Effect of L1 and L2 Regularization on Loss